### 完整版Platt SMO算法

In [1]:
import numpy as np
import pandas as pd

In [2]:
class optStruct:
    '''
    存储相关参数的结构体
    '''
    def __init__(self,dataMatIn,classLabels,C,toler):
        '''
        :param dataMatIn:输入数据,shape(m,n)
        :param classLabels:类别标签,shape(1,m)
        :param C:惩罚参数
        :param toler:容错参数
        '''
        self.X=dataMatIn
        self.labelMat=classLabels.reshape((-1,1))
        self.C=C
        self.tol=toler
        self.m=dataMatIn.shape[0]
        self.alpha=np.zeros((self.m,1))
        self.b=0
        self.eCache=np.zeros((self.m,2))  #误差缓存

In [3]:
def calcEk(os,k):
    '''
    计算误差E_k
    :param os:参数的结构体
    :param k:下标
    :return Ek:误差E_k
    '''
    fXk=float(np.dot(np.dot(os.X[k,:],os.X.T),os.alpha*os.labelMat))+os.b
    Ek=fXk-os.labelMat[k]
    return Ek

def clipAlpha(aj,H,L):
    '''
    裁剪alpha
    :param H:上限值
    :param L:下限值
    :return aj:裁剪后的alpha
    '''
    if aj>H:
        aj=H
    if aj<L:
        aj=L
    return aj

def selectJrand(i,m):
    '''
    随机选择一个alpha
    :param i:是第一个alpha的下标
    :param m:是所有alpha的数目
    :return j:随机选择的alpha下标
    '''
    j=i
    while(j==i):
        j=np.random.uniform(0,m)
    return int(j)

def selectJ(i,os,Ei):
    '''
    启发式搜索第二个要优化的alpha,选择使得E_i-E_j最大的alpha值
    :param i:第一个alpha索引
    :param os:参数结构体
    :param Ei:误差E_i
    :return j,Ej:j为第二个alpha索引,误差E_j
    '''
    maxK=-1
    maxDeltaE=0
    Ej=0
    #将输入值Ei在缓存中设置为有效的，这类的有效一维着它已经计算好了
    os.eCache[i]=[1,Ei]
    #找出所有有效的alpha对应的误差
    #这里获取的是行索引
    validEcacheList=np.nonzero(os.eCache[:,0])[0]
    if(len(validEcacheList)>1):
        for k in validEcacheList:
            if k==i:
                continue
            Ek=calcEk(os,k)
            deltaE=abs(Ei-Ek)
            if(deltaE>maxDeltaE):
                maxK=k
                maxDeltaE=deltaE
                Ej=Ek
        #返回使得|E_i-E_j|最大的第二个alpha，及其误差
        return maxK,Ej
    else:
        #第一次循环
        #随机选择第二个alpha
        j=selectJrand(i,os.m)
        Ej=calcEk(os,j)
    return j,Ej

def updateEk(os,k):
    '''
    更新误差缓存
    :param os:参数结构体
    :param k:要更新的索引
    '''
    Ek=calcEk(os,k)
    os.eCache[k]=[1,Ek]

In [4]:
def innerL(i,os):
    '''
    SMO算法的优化更新部分
    :param i:当前索引
    :param os:参数结构体
    '''
    Ei=calcEk(os,i)
    #找第一个不满足KKT条件的alpha
    if ((os.labelMat[i]*Ei<-os.tol) and (os.alpha[i]<os.C)) or \
        ((os.labelMat[i]*Ei>os.tol) and (os.alpha[i]>0)):
        j,Ej=selectJ(i,os,Ei)  #启发式搜索第二个alpha
        alphaIold=os.alpha[i].copy()
        alphaJold=os.alpha[j].copy()
        #计算边界值
        if(os.labelMat[i]!=os.labelMat[j]):
            #y_i!=y_j的情况
            L=max(0,os.alpha[j]-os.alpha[i])
            H=min(os.C,os.C+os.alpha[j]-os.alpha[i])
        else:
            L=max(0,os.alpha[j]+os.alpha[i]-os.C)
            H=min(os.C,os.alpha[j]+os.alpha[i])
        if L==H:
            print("L==H")
            return 0
        #计算eta
        #这里计算的是-eta
        eta=2*np.dot(os.X[i,:],os.X[j,:].T)-np.dot(os.X[i,:],os.X[i,:].T)-\
            np.dot(os.X[j,:],os.X[j,:].T)
        if eta>=0:
            #?
            print("eta>=0")
            return 0
        #更新alpha_1
        os.alpha[j]-=os.labelMat[j]*(Ei-Ej)/eta
        #裁剪alpha_1
        os.alpha[j]=clipAlpha(os.alpha[j],H,L)
        #更新误差缓存
        updateEk(os,j)
        if(abs(os.alpha[j]-alphaJold)<0.00001):
            print("j not moving enough")
            return 0
        #更新alpha_2
        os.alpha[i]+=os.labelMat[j]*os.labelMat[i]*\
                    (alphaJold-os.alpha[j])
        #更新误差缓存
        updateEk(os,i)
        #计算偏差b
        b1=os.b-Ei-os.labelMat[i]*(os.alpha[i]-alphaIold)*(np.dot(os.X[i,:],os.X[i,:].T))- \
            os.labelMat[j]*(os.alpha[j]-alphaJold)*(np.dot(os.X[i,:],os.X[j,:].T))
        b2=os.b-Ej-os.labelMat[i]*(os.alpha[i]-alphaIold)*(np.dot(os.X[i,:],os.X[j,:].T))- \
            os.labelMat[j]*(os.alpha[j]-alphaJold)*(np.dot(os.X[j,:],os.X[j,:].T))
        if(0<os.alpha[i] and os.alpha[i]<os.C):
            os.b=b1
        elif(0<os.alpha[j] and os.alpha[j]<os.C):
            os.b=b2
        else:
            os.b=(b1+b2)/2.0
        return 1
    else:
        return 0

In [5]:
#完整版SMO算法
def smoP(dataMatIn,classLabel,C,toler,maxIter,kTup=('lin',0)):
    os=optStruct(dataMatIn,classLabel,C,toler)
    iter=0
    entireSet=True
    alphaPairsChanged=0
    #当迭代次数超过指定的最大值，或者遍历整个集合都为对任意alpha对进行修改时
    #退出循环
    while((iter<maxIter) and ((alphaPairsChanged>0) or entireSet) ):
        alphaPairsChanged=0
        if entireSet:
            #遍历整个数据集
            for i in range(os.m):
                alphaPairsChanged+=innerL(i,os)
            print("fullSet,iter:%d i:%d,pairs changed %d"%\
                 (iter,i,alphaPairsChanged))
            iter+=1
        else:
            #遍历非边界值,也就是不在边界0或c上的值
            nonBoundIs=np.nonzero((os.alpha>0)*(os.alpha<C))[0]
            for i in nonBoundIs:
                alphaPairsChanged+=innerL(i,os)
            print("fullSet,iter:%d i:%d,pairs changed %d"%\
                 (iter,i,alphaPairsChanged))
            iter+=1
        if entireSet:
            entireSet=False
        elif(alphaPairsChanged==0):
            entireSet=True
    print("iteration number:%d"%iter)
    return os.b,os.alpha

In [6]:
#测试
#导入数据
dataset=pd.read_csv('datasets/Social_Network_Ads.csv')
X=dataset.iloc[:,[2,3]].values
Y=dataset.iloc[:,[4]].values
#这里采用的类别标签是-1和1，所以将标签0转为-1
Y[Y==0]=-1
#划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
#标准化
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [7]:
b,alpha=smoP(X_train,Y_train,0.6,0.001,40)

j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H


In [8]:
print(b,alpha)

[-0.20866678] [[0.6       ]
 [0.6       ]
 [0.        ]
 [0.        ]
 [0.37424879]
 [0.19789583]
 [0.28971434]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.6       ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.6       ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.6       ]
 [0.        ]
 [0.        ]
 [0.30200891]
 [0.6       ]
 [0.        ]
 [0.        ]
 [0.6       ]
 [0.4389147 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.44305931]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.6       ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.  